## Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

## Import data

In [2]:
df = pd.read_csv("online_transaction.csv")
df.head()

,transaction_id,customer_id,transaction_date,product_id,product_name,product_category,quantity,price
0,16679,17850,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71
1,16680,17850,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71
2,16681,17850,1/1/2019,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05
3,16682,17850,1/1/2019,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53
4,16682,17850,1/1/2019,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    52924 non-null  int64  
 1   customer_id       52924 non-null  int64  
 2   transaction_date  52924 non-null  object 
 3   product_id        52924 non-null  object 
 4   product_name      52924 non-null  object 
 5   product_category  52924 non-null  object 
 6   quantity          52924 non-null  int64  
 7   price             52924 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 3.2+ MB


## Data Cleansing

In [4]:
df_clean = df.copy()
# Create column date
df_clean["transaction_date"] = pd.to_datetime(df_clean["transaction_date"]).dt.date.astype('datetime64[ns]')
# make all product_names lowercase
df_clean["product_name"] = df_clean["product_name"].str.lower()
# create an amount value, namely the product of quantity and price
df_clean["amount"] = df_clean["quantity"]*df_clean["price"]
# Replace the product name of a product_id that has several product_names with one of the product_names that appears most frequently
most_freq_product_name = df_clean.groupby(['product_id','product_name'], as_index=False).agg(order_cnt=('transaction_id','nunique')).sort_values(['product_id','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_id')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_id')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# convert customer_id and transaction_id to string
df_clean["customer_id"] = df_clean["customer_id"].astype(str)
df_clean["transaction_id"] = df_clean["transaction_id"].astype(str)

In [5]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    52924 non-null  object        
 1   customer_id       52924 non-null  object        
 2   transaction_date  52924 non-null  datetime64[ns]
 3   product_id        52924 non-null  object        
 4   product_name      52924 non-null  object        
 5   product_category  52924 non-null  object        
 6   quantity          52924 non-null  int64         
 7   price             52924 non-null  float64       
 8   amount            52924 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 3.6+ MB


## Create RFM Segmentation

### Aggregate transaction data into a summary form of total transactions (orders), total order value (order value), last order date for each user

In [6]:
df_user = df_clean.groupby("customer_id", as_index=False).agg(order_cnt=("transaction_id", "nunique"),
                                                             max_order_date=("transaction_date","max"),
                                                             total_order_value=("amount","sum"))
df_user

,customer_id,order_cnt,max_order_date,total_order_value
0,12346,1,2019-09-15,30.99
1,12347,31,2019-11-02,13834.90
2,12348,8,2019-10-19,1442.12
3,12350,11,2019-12-14,1360.07
4,12356,13,2019-09-15,1442.47
...,...,...,...,...
1463,18259,3,2019-04-05,544.34
1464,18260,19,2019-10-05,2363.05
1465,18269,2,2019-06-20,101.56
1466,18277,1,2019-10-23,298.00


## Create a column for the number of days since the last order

In [7]:
today = df_clean["transaction_date"].max()
df_user["day_since_last_order"] = (today-df_user["max_order_date"]).dt.days
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346,1,2019-09-15,30.99,107
1,12347,31,2019-11-02,13834.90,59
2,12348,8,2019-10-19,1442.12,73
3,12350,11,2019-12-14,1360.07,17
4,12356,13,2019-09-15,1442.47,107
...,...,...,...,...,...
1463,18259,3,2019-04-05,544.34,270
1464,18260,19,2019-10-05,2363.05,87
1465,18269,2,2019-06-20,101.56,194
1466,18277,1,2019-10-23,298.00,69


In [8]:
df_user.describe()

,order_cnt,max_order_date,total_order_value,day_since_last_order
count,1468.000000,1468,1468.000000,1468.000000
mean,18.141008,2019-08-08 16:59:10.953678336,3181.740204,144.292234
min,1.000000,2019-01-01 00:00:00,1.000000,0.000000
25%,5.000000,2019-05-25 00:00:00,652.412500,55.000000
50%,11.000000,2019-08-22 00:00:00,1750.410000,131.000000
75%,23.000000,2019-11-06 00:00:00,3917.887500,220.000000
max,328.000000,2019-12-31 00:00:00,75937.550000,364.000000
std,24.976414,NaN,5065.804553,101.936959


## Create a binning of the number of days since the last order consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label 1 to 5 from highest to lowest bin as recency score

In [9]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346,1,2019-09-15,30.99,107,3
1,12347,31,2019-11-02,13834.90,59,4
2,12348,8,2019-10-19,1442.12,73,4
3,12350,11,2019-12-14,1360.07,17,5
4,12356,13,2019-09-15,1442.47,107,3
...,...,...,...,...,...,...
1463,18259,3,2019-04-05,544.34,270,1
1464,18260,19,2019-10-05,2363.05,87,4
1465,18269,2,2019-06-20,101.56,194,2
1466,18277,1,2019-10-23,298.00,69,4


In [10]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           1468 non-null   object        
 1   order_cnt             1468 non-null   int64         
 2   max_order_date        1468 non-null   datetime64[ns]
 3   total_order_value     1468 non-null   float64       
 4   day_since_last_order  1468 non-null   int64         
 5   recency_score         1468 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 63.2+ KB


## Create a binning of total transactions (orders) consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label them 1 to 5 from lowest to highest bin as the frequency score

In [11]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[df_user['order_cnt'].min(),
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12346,1,2019-09-15,30.99,107,3,1
1,12347,31,2019-11-02,13834.90,59,4,5
2,12348,8,2019-10-19,1442.12,73,4,2
3,12350,11,2019-12-14,1360.07,17,5,3
4,12356,13,2019-09-15,1442.47,107,3,3
...,...,...,...,...,...,...,...
1463,18259,3,2019-04-05,544.34,270,1,1
1464,18260,19,2019-10-05,2363.05,87,4,4
1465,18269,2,2019-06-20,101.56,194,2,1
1466,18277,1,2019-10-23,298.00,69,4,1


In [12]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           1468 non-null   object        
 1   order_cnt             1468 non-null   int64         
 2   max_order_date        1468 non-null   datetime64[ns]
 3   total_order_value     1468 non-null   float64       
 4   day_since_last_order  1468 non-null   int64         
 5   recency_score         1468 non-null   int32         
 6   frequency_score       1468 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), object(1)
memory usage: 68.9+ KB


## Create a binning of the total order value (order value) consisting of 5 bins with the boundaries being min, P20, P40, P60, P80, max and label them 1 to 5 from lowest to highest bin as a monetary score

In [13]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12346,1,2019-09-15,30.99,107,3,1,1
1,12347,31,2019-11-02,13834.90,59,4,5,5
2,12348,8,2019-10-19,1442.12,73,4,2,3
3,12350,11,2019-12-14,1360.07,17,5,3,3
4,12356,13,2019-09-15,1442.47,107,3,3,3
...,...,...,...,...,...,...,...,...
1463,18259,3,2019-04-05,544.34,270,1,1,2
1464,18260,19,2019-10-05,2363.05,87,4,4,3
1465,18269,2,2019-06-20,101.56,194,2,1,1
1466,18277,1,2019-10-23,298.00,69,4,1,1


In [14]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           1468 non-null   object        
 1   order_cnt             1468 non-null   int64         
 2   max_order_date        1468 non-null   datetime64[ns]
 3   total_order_value     1468 non-null   float64       
 4   day_since_last_order  1468 non-null   int64         
 5   recency_score         1468 non-null   int32         
 6   frequency_score       1468 non-null   int32         
 7   monetary_score        1468 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(1)
memory usage: 74.7+ KB


## Create a segment name column based on recency and frequency scores

In [15]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score,segment
0,12346,1,2019-09-15,30.99,107,3,1,1,09-About to Sleep
1,12347,31,2019-11-02,13834.90,59,4,5,5,02-Loyal Customers
2,12348,8,2019-10-19,1442.12,73,4,2,3,03-Potential Loyalists
3,12350,11,2019-12-14,1360.07,17,5,3,3,03-Potential Loyalists
4,12356,13,2019-09-15,1442.47,107,3,3,3,05-Need Attention
...,...,...,...,...,...,...,...,...,...
1463,18259,3,2019-04-05,544.34,270,1,1,2,10-Hibernating
1464,18260,19,2019-10-05,2363.05,87,4,4,3,02-Loyal Customers
1465,18269,2,2019-06-20,101.56,194,2,1,1,10-Hibernating
1466,18277,1,2019-10-23,298.00,69,4,1,1,07-Promising


In [16]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           1468 non-null   object        
 1   order_cnt             1468 non-null   int64         
 2   max_order_date        1468 non-null   datetime64[ns]
 3   total_order_value     1468 non-null   float64       
 4   day_since_last_order  1468 non-null   int64         
 5   recency_score         1468 non-null   int32         
 6   frequency_score       1468 non-null   int32         
 7   monetary_score        1468 non-null   int32         
 8   segment               1468 non-null   object        
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(2)
memory usage: 86.1+ KB


## Display a summary of RFM segmentation (point 8) in the form of number of users, average and median of total orders, total order value, and number of days since the last order

# Conclusion and recommendation

In [17]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'customer_id': pd.Series.nunique,
                        'day_since_last_order': [np.mean, np.median],
                        'order_cnt': [np.mean, np.median],
                        'total_order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

customer_id day_since_last_order         order_cnt  \
                           nunique                 mean median       mean   
segment                                                                     
01-Champion                    154            21.480519   20.5  46.311688   
02-Loyal Customers             249            92.658635   89.0  35.441767   
03-Potential Loyalists         214            43.612150   34.0   9.369159   
04-Can't Lose Them              66           249.984848  245.5  41.742424   
05-Need Attention               59           131.847458  131.0  11.627119   
06-New Customers                33            18.484848   19.0   2.151515   
07-Promising                    42            74.738095   76.0   2.214286   
08-At Risk                     203           247.901478  240.0  16.068966   
09-About to Sleep              130           132.400000  134.0   4.146154   
10-Hibernating                 318           253.286164  256.0   3.971698   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              33.0       8959.054351  6099.420       10.5  
02-Loyal Customers       28.0       6162.167149  4590.740       17.0  
03-Potential Loyalists    9.0       1610.597570  1449.075       14.6  
04-Can't Lose Them       36.5       7034.938333  6415.535        4.5  
05-Need Attention        11.0       1795.358136  1564.210        4.0  
06-New Customers          2.0        411.342727   362.870        2.2  
07-Promising              2.0        485.016905   305.995        2.9  
08-At Risk               14.0       2673.189557  2278.900       13.8  
09-About to Sleep         4.0        543.706077   479.160        8.9  
10-Hibernating            4.0        611.749686   485.080       21.7

# Conclusion and recommendation

* The most users are in the Hibernating segment (318 or 21.7%), Loyal Customers (249 or 17%), and Potential Loyalists (214 or 14.6%).
* Creating a special program to increase the number of transactions for Loyal Customers so that customers in that segment can increase their transaction frequency so they can move up to the Champion segment.
* Developing a special program that focuses on the urgency of transactions for customers in the Potential Loyalists segment so that customers in that segment will transact again in the near future so they can move up to the Champion segment.
* Creation of a special program for customers in the Hibernating segment so that customers in that segment will immediately make transactions again even though the transaction frequency is still small so that they become customers in the New Customers segment.